In [1]:
import client_lib
import shlex

In [2]:
work_dir = "/home/jovyan/vmeshchaninov/LatentDiffusion"

# Encoder training

In [11]:
# reconstruction training on wikipedia-512

num_latents = 1

script = f"{work_dir}/train_encoder.py " \
         f"dataset=wikipedia-512 " \
         f"encoder.latent.num_latents={num_latents} " \
         f"decoder.latent.num_latents={num_latents} " \
         f"encoder.augmentation.masking.weight=0. " \
         f"encoder.augmentation.gaussian_noise.weight=0. "


In [3]:
# reconstruction training on wikipedia

num_latents = 16
suffix = "v2.0"

script = f"{work_dir}/train_encoder.py " \
         f"dataset=wikipedia " \
         f"encoder.latent.num_latents={num_latents} " \
         f"decoder.latent.num_latents={num_latents} " \
         f"encoder.augmentation.masking.weight=0. " \
         f"encoder.augmentation.gaussian_noise.weight=0. " \
         f"suffix={suffix}"


In [8]:
# v1.1 training on wikipedia

num_latents = 64

script = f"{work_dir}/train_encoder.py " \
         f"dataset=wikipedia " \
         f"encoder.latent.num_latents={num_latents} " \
         f"decoder.latent.num_latents={num_latents} " \
         f"encoder.augmentation.masking.weight=0.5 " \
         f"encoder.augmentation.gaussian_noise.weight=0.5 "


In [33]:
# masking ablation

ratio = 1
num_latents = 16
suffix = f"v2.1.{ratio}"

# checkpoint = "autoencoder-num_latents=16-wikipedia-v2.1.5-128/80000.pth"

script = f"{work_dir}/train_encoder.py " \
         f"dataset=wikipedia " \
         f"encoder.latent.num_latents={num_latents} " \
         f"decoder.latent.num_latents={num_latents} " \
         f"encoder.augmentation.masking.weight=1.0 " \
         f"encoder.augmentation.masking.encodings_mlm_probability={ratio / 10.} " \
         f"encoder.augmentation.gaussian_noise.weight=0. " \
         f"suffix={suffix}"

In [14]:
# gaussian ablation

num_latents = 128
suffix = f"final"

script = f"{work_dir}/train_encoder.py " \
         f"dataset=wikipedia " \
         f"encoder.latent.num_latents={num_latents} " \
         f"decoder.latent.num_latents={num_latents} " \
         f"suffix={suffix}"

In [15]:
N_GPUS = 8
N_WORKERS = 2

base_image = "cr.ai.cloud.ru/6e6f5587-f3c6-4c5d-b535-0c4fce4c6027/jupyter-job-textdif:0.8"
EXP_NAME = 'autoencoder'


job = client_lib.Job(
    job_desc=f"#other#RnD-meshchaninov-{EXP_NAME}",
    base_image=base_image,
    script=script,
    n_workers = N_WORKERS,
    instance_type=f"a100.{N_GPUS}gpu",
    type = 'pytorch2',
    preflight_check=True,
    env_variables={
        'NCCL_IB_SL': "",
        'NCCL_P2P_LEVEL': "",
        'NCCL_NET_GDR_LEVEL': "",
        'NCCL_MAX_NRINGS': "",
        'PYTHONPATH': "",
        'PROJECT_ROOT': work_dir,
    },
    region='A100-MT',
    flags = {},
)

job.submit()

/home/user/conda/lib/python3.11/site-packages/client_lib.py:412: DeprecationWarning: The `preflight_check` parameter is deprecated and has no effect. Soon the parameter support will stop completely, please do not use it.
  warnings.warn(msg, DeprecationWarning)


'Job "lm-mpi-job-02696755-f47c-48e2-ae99-93244d14bc34" created.'

In [13]:
job.status()

'Job status=Failed'

In [12]:
job.kill()

'Job "lm-mpi-job-c30acc3a-cc73-45e8-bf15-a4b33cab7338" deleted'

## OpenWebText

In [8]:
# gaussian ablation

num_latents = 128
suffix = f"v1.0"

script = f"{work_dir}/train_encoder.py " \
         f"dataset=openwebtext-512 " \
         f"project.name='latent-diffusion-article-large-v1.0' " \
         f"encoder.latent.num_latents={num_latents} " \
         f"encoder.embedding.max_position_embeddings=512 " \
         f"decoder.latent.num_latents={num_latents} " \
         f"decoder.embedding.max_position_embeddings=512 " \
         f"autoencoder.training.training_iters=100000 " \
         f"suffix={suffix}"

In [9]:
N_GPUS = 8
N_WORKERS = 2

base_image = "cr.ai.cloud.ru/6e6f5587-f3c6-4c5d-b535-0c4fce4c6027/jupyter-job-textdif:0.8"
EXP_NAME = 'autoencoder'


job = client_lib.Job(
    job_desc=f"#other#RnD-meshchaninov-{EXP_NAME}",
    base_image=base_image,
    script=script,
    n_workers = N_WORKERS,
    instance_type=f"a100.{N_GPUS}gpu",
    type = 'pytorch2',
    preflight_check=True,
    env_variables={
        'NCCL_IB_SL': "",
        'NCCL_P2P_LEVEL': "",
        'NCCL_NET_GDR_LEVEL': "",
        'NCCL_MAX_NRINGS': "",
        'PYTHONPATH': "",
        'PROJECT_ROOT': work_dir,
    },
    region='A100-MT',
    flags = {},
)

job.submit()

/home/user/conda/lib/python3.11/site-packages/client_lib.py:412: DeprecationWarning: The `preflight_check` parameter is deprecated and has no effect. Soon the parameter support will stop completely, please do not use it.
  warnings.warn(msg, DeprecationWarning)


'Job "lm-mpi-job-1a03c349-6fa5-4171-a80b-7ea93c6e4440" created.'

## Finetuning

In [5]:
def make_checkpoint(name):
    list_ = []
    for s in list(name):
        if s in ['[', ']', '(', ')', '=', ',']:
            list_.append(f"\\{s}")
            
        else:
            list_.append(s)
    t = "".join(list_)
    print(list_)
    print(t)
    return t

In [6]:
num_latents = 16
suffix = "v1.1-simclr_finetune-temp_10"

checkpoint = "autoencoder-num_latents=16-wikipedia-v1.1-128/100000.pth"

script = f"{work_dir}/train_encoder.py " \
         f"dataset=wikipedia " \
         f"encoder.latent.num_latents={num_latents} " \
         f"decoder.latent.num_latents={num_latents} " \
         f"encoder.augmentation.masking.weight=0.5 " \
         f"encoder.augmentation.gaussian_noise.weight=0.5 " \
         f"autoencoder.training.training_iters=110000 " \
         f"autoencoder.logging.eval_freq=10000 " \
         f"autoencoder.model.load_checkpoint='{make_checkpoint(checkpoint)}' " \
         f"suffix={suffix} "


['a', 'u', 't', 'o', 'e', 'n', 'c', 'o', 'd', 'e', 'r', '-', 'n', 'u', 'm', '_', 'l', 'a', 't', 'e', 'n', 't', 's', '\\=', '1', '6', '-', 'w', 'i', 'k', 'i', 'p', 'e', 'd', 'i', 'a', '-', 'v', '1', '.', '1', '-', '1', '2', '8', '/', '1', '0', '0', '0', '0', '0', '.', 'p', 't', 'h']
autoencoder-num_latents\=16-wikipedia-v1.1-128/100000.pth


In [7]:
N_GPUS = 8
N_WORKERS = 2

base_image = "cr.ai.cloud.ru/6e6f5587-f3c6-4c5d-b535-0c4fce4c6027/jupyter-job-textdif:0.8"
EXP_NAME = 'autoencoder'


job = client_lib.Job(
    job_desc=f"#other#RnD-meshchaninov-{EXP_NAME}",
    base_image=base_image,
    script=script,
    n_workers = N_WORKERS,
    instance_type=f"a100.{N_GPUS}gpu",
    type = 'pytorch2',
    preflight_check=True,
    env_variables={
        'NCCL_IB_SL': "",
        'NCCL_P2P_LEVEL': "",
        'NCCL_NET_GDR_LEVEL': "",
        'NCCL_MAX_NRINGS': "",
        'PYTHONPATH': "",
        'PROJECT_ROOT': work_dir,
    },
    region='A100-MT',
    flags = {},
)

job.submit()

/home/user/conda/lib/python3.11/site-packages/client_lib.py:412: DeprecationWarning: The `preflight_check` parameter is deprecated and has no effect. Soon the parameter support will stop completely, please do not use it.
  warnings.warn(msg, DeprecationWarning)


'Job "lm-mpi-job-2694114b-61b8-41e9-ab4d-2d8000e23aef" created.'

## Cycle

In [10]:
# reconstruction training on wikipedia

suffix = "v1.0"

for num_latents in [2, 4, 8, 16, 32, 64]:

    script = f"{work_dir}/train_encoder.py " \
            f"dataset=wikipedia " \
            f"encoder.latent.num_latents={num_latents} " \
            f"decoder.latent.num_latents={num_latents} " \
            f"encoder.augmentation.masking.weight=0. " \
            f"encoder.augmentation.gaussian_noise.weight=0. " \
            f"suffix={suffix}"

    N_GPUS = 8
    N_WORKERS = 1

    base_image = "cr.ai.cloud.ru/6e6f5587-f3c6-4c5d-b535-0c4fce4c6027/jupyter-job-textdif:0.8"
    EXP_NAME = 'autoencoder'


    job = client_lib.Job(
        job_desc=f"#other#RnD-meshchaninov-{EXP_NAME}",
        base_image=base_image,
        script=script,
        n_workers = N_WORKERS,
        instance_type=f"a100.{N_GPUS}gpu",
        type = 'pytorch2',
        preflight_check=True,
        env_variables={
            'NCCL_IB_SL': "",
            'NCCL_P2P_LEVEL': "",
            'NCCL_NET_GDR_LEVEL': "",
            'NCCL_MAX_NRINGS': "",
            'PYTHONPATH': "",
            'PROJECT_ROOT': work_dir,
        },
        region='A100-MT',
        flags = {},
    )

    job.submit()

/home/user/conda/lib/python3.11/site-packages/client_lib.py:412: DeprecationWarning: The `preflight_check` parameter is deprecated and has no effect. Soon the parameter support will stop completely, please do not use it.
  warnings.warn(msg, DeprecationWarning)
/home/user/conda/lib/python3.11/site-packages/client_lib.py:412: DeprecationWarning: The `preflight_check` parameter is deprecated and has no effect. Soon the parameter support will stop completely, please do not use it.
  warnings.warn(msg, DeprecationWarning)
/home/user/conda/lib/python3.11/site-packages/client_lib.py:412: DeprecationWarning: The `preflight_check` parameter is deprecated and has no effect. Soon the parameter support will stop completely, please do not use it.
  warnings.warn(msg, DeprecationWarning)
/home/user/conda/lib/python3.11/site-packages/client_lib.py:412: DeprecationWarning: The `preflight_check` parameter is deprecated and has no effect. Soon the parameter support will stop completely, please do not u

# Diffusion Training

In [3]:
def make_checkpoint(name):
    list_ = []
    for s in list(name):
        if s in ['[', ']', '(', ')', '=', ',']:
            list_.append(f"\\{s}")
            
        else:
            list_.append(s)
    t = "".join(list_)
    print(list_)
    print(t)
    return t

In [4]:
num_latents = 8
suffix = "final"

checkpoint = f"autoencoder-num_latents={num_latents}-wikipedia-{suffix}-128/100000.pth"

script = f"{work_dir}/train_diffusion.py " \
         f"training=diffusion " \
         f"dataset=rocstories " \
         f"autoencoder.model.load_checkpoint='{make_checkpoint(checkpoint)}' " \
         f"encoder.latent.num_latents={num_latents} " \
         f"decoder.latent.num_latents={num_latents} " \
         f"diffusion.dynamic.d=7 " \
         f"suffix={suffix}-d7 "

['a', 'u', 't', 'o', 'e', 'n', 'c', 'o', 'd', 'e', 'r', '-', 'n', 'u', 'm', '_', 'l', 'a', 't', 'e', 'n', 't', 's', '\\=', '8', '-', 'w', 'i', 'k', 'i', 'p', 'e', 'd', 'i', 'a', '-', 'f', 'i', 'n', 'a', 'l', '-', '1', '2', '8', '/', '1', '0', '0', '0', '0', '0', '.', 'p', 't', 'h']
autoencoder-num_latents\=8-wikipedia-final-128/100000.pth


In [5]:
N_GPUS = 8
N_WORKERS = 2

base_image = "cr.ai.cloud.ru/6e6f5587-f3c6-4c5d-b535-0c4fce4c6027/jupyter-job-textdif:0.8"
EXP_NAME = 'autoencoder'


job = client_lib.Job(
    job_desc=f"#other#RnD-meshchaninov-{EXP_NAME}",
    base_image=base_image,
    script=script,
    n_workers = N_WORKERS,
    instance_type=f"a100.{N_GPUS}gpu",
    type = 'pytorch2',
    preflight_check=True,
    env_variables={
        'NCCL_IB_SL': "",
        'NCCL_P2P_LEVEL': "",
        'NCCL_NET_GDR_LEVEL': "",
        'NCCL_MAX_NRINGS': "",
        'PYTHONPATH': "",
        'PROJECT_ROOT': work_dir,
    },
    region='A100-MT',
    flags = {},
)

job.submit()

/home/user/conda/lib/python3.11/site-packages/client_lib.py:412: DeprecationWarning: The `preflight_check` parameter is deprecated and has no effect. Soon the parameter support will stop completely, please do not use it.
  warnings.warn(msg, DeprecationWarning)


'Job "lm-mpi-job-14ad87e7-0a40-494a-8e70-ec36789d372b" created.'

In [7]:
job.status()

'Job status=Running'

## Wikipedia

In [9]:
num_latents = 128
suffix = f"v1.0"

# checkpoint_dir = f"/home/jovyan/shares/SR004.nfs2/vmeshchaninov/vmeshchaninov/LatentDiffusion/checkpoints_05_07"
checkpoint_dir = "/home/jovyan/vmeshchaninov/LatentDiffusion/checkpoints"
checkpoint = f"autoencoder-num_latents={num_latents}-wikipedia-{suffix}-128/100000.pth"

script = f"{work_dir}/train_diffusion.py " \
         f"training=diffusion " \
         f"dataset=wikipedia " \
         f"project.name='latent-diffusion-article-large-v2.0' " \
         f"project.checkpoint_dir='{make_checkpoint(checkpoint_dir)}' " \
         f"encoder.latent.num_latents={num_latents} " \
         f"encoder.embedding.max_position_embeddings=128 " \
         f"decoder.latent.num_latents={num_latents} " \
         f"decoder.embedding.max_position_embeddings=128 " \
         f"autoencoder.model.load_checkpoint='{make_checkpoint(checkpoint)}' " \
         f"diffusion.dynamic.d=5 " \
         f"diffusion.model.num_workers=10 " \
         f"diffusion.logging.eval_freq=100000 " \
         f"diffusion.logging.save_freq=100000 " \
         f"diffusion.training.training_iters=500000 " \
         f"suffix={suffix}"

['/', 'h', 'o', 'm', 'e', '/', 'j', 'o', 'v', 'y', 'a', 'n', '/', 'v', 'm', 'e', 's', 'h', 'c', 'h', 'a', 'n', 'i', 'n', 'o', 'v', '/', 'L', 'a', 't', 'e', 'n', 't', 'D', 'i', 'f', 'f', 'u', 's', 'i', 'o', 'n', '/', 'c', 'h', 'e', 'c', 'k', 'p', 'o', 'i', 'n', 't', 's']
/home/jovyan/vmeshchaninov/LatentDiffusion/checkpoints
['a', 'u', 't', 'o', 'e', 'n', 'c', 'o', 'd', 'e', 'r', '-', 'n', 'u', 'm', '_', 'l', 'a', 't', 'e', 'n', 't', 's', '\\=', '1', '2', '8', '-', 'w', 'i', 'k', 'i', 'p', 'e', 'd', 'i', 'a', '-', 'v', '1', '.', '0', '-', '1', '2', '8', '/', '1', '0', '0', '0', '0', '0', '.', 'p', 't', 'h']
autoencoder-num_latents\=128-wikipedia-v1.0-128/100000.pth


In [10]:
N_GPUS = 8
N_WORKERS = 2

base_image = "cr.ai.cloud.ru/6e6f5587-f3c6-4c5d-b535-0c4fce4c6027/jupyter-job-textdif:0.8"
EXP_NAME = f'autoencoder-{suffix}-{num_latents}'


job = client_lib.Job(
    job_desc=f"#other#RnD-meshchaninov-{EXP_NAME}",
    base_image=base_image,
    script=script,
    n_workers = N_WORKERS,
    instance_type=f"a100.{N_GPUS}gpu",
    type = 'pytorch2',
    preflight_check=True,
    env_variables={
        'NCCL_IB_SL': "",
        'NCCL_P2P_LEVEL': "",
        'NCCL_NET_GDR_LEVEL': "",
        'NCCL_MAX_NRINGS': "",
        'PYTHONPATH': "",
        'PROJECT_ROOT': work_dir,
    },
    region='A100-MT',
    flags = {},
)

job.submit()

/home/user/conda/lib/python3.11/site-packages/client_lib.py:412: DeprecationWarning: The `preflight_check` parameter is deprecated and has no effect. Soon the parameter support will stop completely, please do not use it.
  warnings.warn(msg, DeprecationWarning)


'Job "lm-mpi-job-4dd7c994-e3ec-4e65-94e3-24eb6dca711f" created.'

In [6]:
job.kill()

'Job "lm-mpi-job-0214961b-afb9-4626-896b-0a8f313771bd" deleted'

## OpenWebText

In [13]:
num_latents = 512
suffix = f"v1.0"

checkpoint = f"autoencoder-num_latents={num_latents}-openwebtext-512-{suffix}-512/100000.pth"

script = f"{work_dir}/train_diffusion.py " \
         f"training=diffusion " \
         f"dataset=openwebtext-512 " \
         f"project.name='latent-diffusion-article-large-v1.0' " \
         f"encoder.latent.num_latents={num_latents} " \
         f"encoder.embedding.max_position_embeddings=512 " \
         f"decoder.latent.num_latents={num_latents} " \
         f"decoder.embedding.max_position_embeddings=512 " \
         f"autoencoder.model.load_checkpoint='{make_checkpoint(checkpoint)}' " \
         f"diffusion.dynamic.d=3 " \
         f"diffusion.model.num_workers=10 " \
         f"suffix={suffix}"

['a', 'u', 't', 'o', 'e', 'n', 'c', 'o', 'd', 'e', 'r', '-', 'n', 'u', 'm', '_', 'l', 'a', 't', 'e', 'n', 't', 's', '\\=', '5', '1', '2', '-', 'o', 'p', 'e', 'n', 'w', 'e', 'b', 't', 'e', 'x', 't', '-', '5', '1', '2', '-', 'v', '1', '.', '0', '-', '5', '1', '2', '/', '1', '0', '0', '0', '0', '0', '.', 'p', 't', 'h']
autoencoder-num_latents\=512-openwebtext-512-v1.0-512/100000.pth


In [14]:
N_GPUS = 8
N_WORKERS = 2

base_image = "cr.ai.cloud.ru/6e6f5587-f3c6-4c5d-b535-0c4fce4c6027/jupyter-job-textdif:0.8"
EXP_NAME = 'autoencoder'


job = client_lib.Job(
    job_desc=f"#other#RnD-meshchaninov-{EXP_NAME}",
    base_image=base_image,
    script=script,
    n_workers = N_WORKERS,
    instance_type=f"a100.{N_GPUS}gpu",
    type = 'pytorch2',
    preflight_check=True,
    env_variables={
        'NCCL_IB_SL': "",
        'NCCL_P2P_LEVEL': "",
        'NCCL_NET_GDR_LEVEL': "",
        'NCCL_MAX_NRINGS': "",
        'PYTHONPATH': "",
        'PROJECT_ROOT': work_dir,
    },
    region='A100-MT',
    flags = {},
)

job.submit()

/home/user/conda/lib/python3.11/site-packages/client_lib.py:412: DeprecationWarning: The `preflight_check` parameter is deprecated and has no effect. Soon the parameter support will stop completely, please do not use it.
  warnings.warn(msg, DeprecationWarning)


'Job "lm-mpi-job-e4919f79-f2e1-4c1b-b7d1-35f4010d9b96" created.'

In [10]:
job.kill()

'Job "lm-mpi-job-89e9a499-df99-428e-8f3c-e20b1125e994" deleted'

# Decoder finetuning

In [3]:
def make_checkpoint(name):
    list_ = []
    for s in list(name):
        if s in ['[', ']', '(', ')', '=', ',']:
            list_.append(f"\\{s}")
            
        else:
            list_.append(s)
    t = "".join(list_)
    print(list_)
    print(t)
    return t

In [4]:
num_latents = 16
suffix = "v2.5.6.7"

checkpoint = "autoencoder-num_latents=16-wikipedia-v2.3.4-128/100000.pth"

script = f"{work_dir}/finetune_decoder.py " \
         f"dataset=wikipedia " \
         f"autoencoder.model.load_checkpoint='{make_checkpoint(checkpoint)}' " \
         f"decoder.finetuning.alpha=0.7 " \
         f"decoder.finetuning.is_alpha=true " \
         f"decoder.finetuning.bert_output_masking=true " \
         f"decoder.finetuning.latent_masking=true " \
         f"suffix={suffix} "

['a', 'u', 't', 'o', 'e', 'n', 'c', 'o', 'd', 'e', 'r', '-', 'n', 'u', 'm', '_', 'l', 'a', 't', 'e', 'n', 't', 's', '\\=', '1', '6', '-', 'w', 'i', 'k', 'i', 'p', 'e', 'd', 'i', 'a', '-', 'v', '2', '.', '3', '.', '4', '-', '1', '2', '8', '/', '1', '0', '0', '0', '0', '0', '.', 'p', 't', 'h']
autoencoder-num_latents\=16-wikipedia-v2.3.4-128/100000.pth


In [5]:
N_GPUS = 8
N_WORKERS = 1

base_image = "cr.ai.cloud.ru/6e6f5587-f3c6-4c5d-b535-0c4fce4c6027/jupyter-job-textdif:0.8"
EXP_NAME = 'autoencoder'


job = client_lib.Job(
    job_desc=f"#other#RnD-meshchaninov-{EXP_NAME}",
    base_image=base_image,
    script=script,
    n_workers = N_WORKERS,
    instance_type=f"a100.{N_GPUS}gpu",
    type = 'pytorch2',
    preflight_check=True,
    env_variables={
        'NCCL_IB_SL': "",
        'NCCL_P2P_LEVEL': "",
        'NCCL_NET_GDR_LEVEL': "",
        'NCCL_MAX_NRINGS': "",
        'PYTHONPATH': "",
        'PROJECT_ROOT': work_dir,
    },
    region='A100-MT',
    flags = {},
)

job.submit()

/home/user/conda/lib/python3.11/site-packages/client_lib.py:412: DeprecationWarning: The `preflight_check` parameter is deprecated and has no effect. Soon the parameter support will stop completely, please do not use it.
  warnings.warn(msg, DeprecationWarning)


'Job "lm-mpi-job-2afe4cf3-3c0a-4991-8d0c-341b4f01a8f1" created.'

In [4]:
for i in range(2, 8):
    num_latents = 16
    suffix = f"v2.5.3.{i}"

    checkpoint = "autoencoder-num_latents=16-wikipedia-v2.3.4-128/100000.pth"

    script = f"{work_dir}/finetune_decoder.py " \
            f"dataset=wikipedia " \
            f"autoencoder.model.load_checkpoint='{make_checkpoint(checkpoint)}' " \
            f"decoder.finetuning.max_std={i / 10.} " \
            f"decoder.finetuning.is_alpha=true " \
            f"decoder.finetuning.bert_output_masking=false " \
            f"decoder.finetuning.latent_masking=false " \
            f"suffix={suffix} "

    N_GPUS = 4
    N_WORKERS = 1

    base_image = "cr.ai.cloud.ru/6e6f5587-f3c6-4c5d-b535-0c4fce4c6027/jupyter-job-textdif:0.8"
    EXP_NAME = 'autoencoder'


    job = client_lib.Job(
        job_desc=f"#other#RnD-meshchaninov-{EXP_NAME}",
        base_image=base_image,
        script=script,
        n_workers = N_WORKERS,
        instance_type=f"a100.{N_GPUS}gpu",
        type = 'pytorch2',
        preflight_check=True,
        env_variables={
            'NCCL_IB_SL': "",
            'NCCL_P2P_LEVEL': "",
            'NCCL_NET_GDR_LEVEL': "",
            'NCCL_MAX_NRINGS': "",
            'PYTHONPATH': "",
            'PROJECT_ROOT': work_dir,
        },
        region='A100-MT',
        flags = {},
    )

    job.submit()

['a', 'u', 't', 'o', 'e', 'n', 'c', 'o', 'd', 'e', 'r', '-', 'n', 'u', 'm', '_', 'l', 'a', 't', 'e', 'n', 't', 's', '\\=', '1', '6', '-', 'w', 'i', 'k', 'i', 'p', 'e', 'd', 'i', 'a', '-', 'v', '2', '.', '3', '.', '4', '-', '1', '2', '8', '/', '1', '0', '0', '0', '0', '0', '.', 'p', 't', 'h']
autoencoder-num_latents\=16-wikipedia-v2.3.4-128/100000.pth


/home/user/conda/lib/python3.11/site-packages/client_lib.py:412: DeprecationWarning: The `preflight_check` parameter is deprecated and has no effect. Soon the parameter support will stop completely, please do not use it.
  warnings.warn(msg, DeprecationWarning)


['a', 'u', 't', 'o', 'e', 'n', 'c', 'o', 'd', 'e', 'r', '-', 'n', 'u', 'm', '_', 'l', 'a', 't', 'e', 'n', 't', 's', '\\=', '1', '6', '-', 'w', 'i', 'k', 'i', 'p', 'e', 'd', 'i', 'a', '-', 'v', '2', '.', '3', '.', '4', '-', '1', '2', '8', '/', '1', '0', '0', '0', '0', '0', '.', 'p', 't', 'h']
autoencoder-num_latents\=16-wikipedia-v2.3.4-128/100000.pth


/home/user/conda/lib/python3.11/site-packages/client_lib.py:412: DeprecationWarning: The `preflight_check` parameter is deprecated and has no effect. Soon the parameter support will stop completely, please do not use it.
  warnings.warn(msg, DeprecationWarning)


['a', 'u', 't', 'o', 'e', 'n', 'c', 'o', 'd', 'e', 'r', '-', 'n', 'u', 'm', '_', 'l', 'a', 't', 'e', 'n', 't', 's', '\\=', '1', '6', '-', 'w', 'i', 'k', 'i', 'p', 'e', 'd', 'i', 'a', '-', 'v', '2', '.', '3', '.', '4', '-', '1', '2', '8', '/', '1', '0', '0', '0', '0', '0', '.', 'p', 't', 'h']
autoencoder-num_latents\=16-wikipedia-v2.3.4-128/100000.pth


...

/home/user/conda/lib/python3.11/site-packages/client_lib.py:412: DeprecationWarning: The `preflight_check` parameter is deprecated and has no effect. Soon the parameter support will stop completely, please do not use it.
  warnings.warn(msg, DeprecationWarning)


['a', 'u', 't', 'o', 'e', 'n', 'c', 'o', 'd', 'e', 'r', '-', 'n', 'u', 'm', '_', 'l', 'a', 't', 'e', 'n', 't', 's', '\\=', '1', '6', '-', 'w', 'i', 'k', 'i', 'p', 'e', 'd', 'i', 'a', '-', 'v', '2', '.', '3', '.', '4', '-', '1', '2', '8', '/', '1', '0', '0', '0', '0', '0', '.', 'p', 't', 'h']
autoencoder-num_latents\=16-wikipedia-v2.3.4-128/100000.pth


/home/user/conda/lib/python3.11/site-packages/client_lib.py:412: DeprecationWarning: The `preflight_check` parameter is deprecated and has no effect. Soon the parameter support will stop completely, please do not use it.
  warnings.warn(msg, DeprecationWarning)


['a', 'u', 't', 'o', 'e', 'n', 'c', 'o', 'd', 'e', 'r', '-', 'n', 'u', 'm', '_', 'l', 'a', 't', 'e', 'n', 't', 's', '\\=', '1', '6', '-', 'w', 'i', 'k', 'i', 'p', 'e', 'd', 'i', 'a', '-', 'v', '2', '.', '3', '.', '4', '-', '1', '2', '8', '/', '1', '0', '0', '0', '0', '0', '.', 'p', 't', 'h']
autoencoder-num_latents\=16-wikipedia-v2.3.4-128/100000.pth


/home/user/conda/lib/python3.11/site-packages/client_lib.py:412: DeprecationWarning: The `preflight_check` parameter is deprecated and has no effect. Soon the parameter support will stop completely, please do not use it.
  warnings.warn(msg, DeprecationWarning)


# Diffusion Generation

In [3]:
def make_checkpoint(name):
    list_ = []
    for s in list(name):
        if s in ['[', ']', '(', ')', '=', ',']:
            list_.append(f"\\{s}")
            
        else:
            list_.append(s)
    t = "".join(list_)
    print(list_)
    print(t)
    return t

In [4]:
suffix = "final"
num_latents = 32

checkpoint = f"autoencoder-num_latents=32-wikipedia-final-128/100000.pth"
diffusion_checkpoint = "diffusion-rocstories-32-d=5-final/120000.pth"

script = f"{work_dir}/diffusion_generation.py " \
         f"training=None " \
         f"dataset=rocstories " \
         f"encoder.latent.num_latents={num_latents} " \
         f"decoder.latent.num_latents={num_latents} " \
         f"autoencoder.model.load_checkpoint='{make_checkpoint(checkpoint)}' " \
         f"diffusion.model.load_checkpoint='{make_checkpoint(diffusion_checkpoint)}' " \
         f"suffix={suffix} "

['a', 'u', 't', 'o', 'e', 'n', 'c', 'o', 'd', 'e', 'r', '-', 'n', 'u', 'm', '_', 'l', 'a', 't', 'e', 'n', 't', 's', '\\=', '3', '2', '-', 'w', 'i', 'k', 'i', 'p', 'e', 'd', 'i', 'a', '-', 'f', 'i', 'n', 'a', 'l', '-', '1', '2', '8', '/', '1', '0', '0', '0', '0', '0', '.', 'p', 't', 'h']
autoencoder-num_latents\=32-wikipedia-final-128/100000.pth
['d', 'i', 'f', 'f', 'u', 's', 'i', 'o', 'n', '-', 'r', 'o', 'c', 's', 't', 'o', 'r', 'i', 'e', 's', '-', '3', '2', '-', 'd', '\\=', '5', '-', 'f', 'i', 'n', 'a', 'l', '/', '1', '2', '0', '0', '0', '0', '.', 'p', 't', 'h']
diffusion-rocstories-32-d\=5-final/120000.pth


In [5]:
N_GPUS = 8
N_WORKERS = 1

base_image = "cr.ai.cloud.ru/6e6f5587-f3c6-4c5d-b535-0c4fce4c6027/jupyter-job-textdif:0.8"
EXP_NAME = 'autoencoder'


job = client_lib.Job(
    job_desc=f"#other#RnD-meshchaninov-{EXP_NAME}",
    base_image=base_image,
    script=script,
    n_workers = N_WORKERS,
    instance_type=f"a100.{N_GPUS}gpu",
    type = 'pytorch2',
    preflight_check=True,
    env_variables={
        'NCCL_IB_SL': "",
        'NCCL_P2P_LEVEL': "",
        'NCCL_NET_GDR_LEVEL': "",
        'NCCL_MAX_NRINGS': "",
        'PYTHONPATH': "",
        'PROJECT_ROOT': work_dir,
    },
    region='A100-MT',
    flags = {},
)

job.submit()

/home/user/conda/lib/python3.11/site-packages/client_lib.py:412: DeprecationWarning: The `preflight_check` parameter is deprecated and has no effect. Soon the parameter support will stop completely, please do not use it.
  warnings.warn(msg, DeprecationWarning)


'Job "lm-mpi-job-bf80dcc4-88bf-4db9-8515-2dfc1d9fc3c3" created.'

In [29]:
for i in range(3, 11):
    suffix = f"v2.5.1.{i}"

    checkpoint = f"autoencoder-num_latents=16-wikipedia-v2.3.4-128-finetuned-{suffix}/10000.pth"
    diffusion_checkpoint = "diffusion-rocstories-16-d=5-v2.3.4/180000.pth"

    script = f"{work_dir}/diffusion_generation.py " \
            f"training=None " \
            f"dataset=rocstories " \
            f"autoencoder.model.load_checkpoint='{make_checkpoint(checkpoint)}' " \
            f"diffusion.model.load_checkpoint='{make_checkpoint(diffusion_checkpoint)}' " \
            f"suffix={suffix} "


    N_GPUS = 8
    N_WORKERS = 2

    base_image = "cr.ai.cloud.ru/6e6f5587-f3c6-4c5d-b535-0c4fce4c6027/jupyter-job-textdif:0.8"
    EXP_NAME = 'autoencoder'


    job = client_lib.Job(
        job_desc=f"#other#RnD-meshchaninov-{EXP_NAME}",
        base_image=base_image,
        script=script,
        n_workers = N_WORKERS,
        instance_type=f"a100.{N_GPUS}gpu",
        type = 'pytorch2',
        preflight_check=True,
        env_variables={
            'NCCL_IB_SL': "",
            'NCCL_P2P_LEVEL': "",
            'NCCL_NET_GDR_LEVEL': "",
            'NCCL_MAX_NRINGS': "",
            'PYTHONPATH': "",
            'PROJECT_ROOT': work_dir,
        },
        region='A100-MT',
        flags = {},
    )

    job.submit()

['a', 'u', 't', 'o', 'e', 'n', 'c', 'o', 'd', 'e', 'r', '-', 'n', 'u', 'm', '_', 'l', 'a', 't', 'e', 'n', 't', 's', '\\=', '1', '6', '-', 'w', 'i', 'k', 'i', 'p', 'e', 'd', 'i', 'a', '-', 'v', '2', '.', '3', '.', '4', '-', '1', '2', '8', '-', 'f', 'i', 'n', 'e', 't', 'u', 'n', 'e', 'd', '-', 'v', '2', '.', '5', '.', '1', '.', '3', '/', '1', '0', '0', '0', '0', '.', 'p', 't', 'h']
autoencoder-num_latents\=16-wikipedia-v2.3.4-128-finetuned-v2.5.1.3/10000.pth
['d', 'i', 'f', 'f', 'u', 's', 'i', 'o', 'n', '-', 'r', 'o', 'c', 's', 't', 'o', 'r', 'i', 'e', 's', '-', '1', '6', '-', 'd', '\\=', '5', '-', 'v', '2', '.', '3', '.', '4', '/', '1', '8', '0', '0', '0', '0', '.', 'p', 't', 'h']
diffusion-rocstories-16-d\=5-v2.3.4/180000.pth


/home/user/conda/lib/python3.11/site-packages/client_lib.py:412: DeprecationWarning: The `preflight_check` parameter is deprecated and has no effect. Soon the parameter support will stop completely, please do not use it.
  warnings.warn(msg, DeprecationWarning)


['a', 'u', 't', 'o', 'e', 'n', 'c', 'o', 'd', 'e', 'r', '-', 'n', 'u', 'm', '_', 'l', 'a', 't', 'e', 'n', 't', 's', '\\=', '1', '6', '-', 'w', 'i', 'k', 'i', 'p', 'e', 'd', 'i', 'a', '-', 'v', '2', '.', '3', '.', '4', '-', '1', '2', '8', '-', 'f', 'i', 'n', 'e', 't', 'u', 'n', 'e', 'd', '-', 'v', '2', '.', '5', '.', '1', '.', '4', '/', '1', '0', '0', '0', '0', '.', 'p', 't', 'h']
autoencoder-num_latents\=16-wikipedia-v2.3.4-128-finetuned-v2.5.1.4/10000.pth
['d', 'i', 'f', 'f', 'u', 's', 'i', 'o', 'n', '-', 'r', 'o', 'c', 's', 't', 'o', 'r', 'i', 'e', 's', '-', '1', '6', '-', 'd', '\\=', '5', '-', 'v', '2', '.', '3', '.', '4', '/', '1', '8', '0', '0', '0', '0', '.', 'p', 't', 'h']
diffusion-rocstories-16-d\=5-v2.3.4/180000.pth


/home/user/conda/lib/python3.11/site-packages/client_lib.py:412: DeprecationWarning: The `preflight_check` parameter is deprecated and has no effect. Soon the parameter support will stop completely, please do not use it.
  warnings.warn(msg, DeprecationWarning)


['a', 'u', 't', 'o', 'e', 'n', 'c', 'o', 'd', 'e', 'r', '-', 'n', 'u', 'm', '_', 'l', 'a', 't', 'e', 'n', 't', 's', '\\=', '1', '6', '-', 'w', 'i', 'k', 'i', 'p', 'e', 'd', 'i', 'a', '-', 'v', '2', '.', '3', '.', '4', '-', '1', '2', '8', '-', 'f', 'i', 'n', 'e', 't', 'u', 'n', 'e', 'd', '-', 'v', '2', '.', '5', '.', '1', '.', '5', '/', '1', '0', '0', '0', '0', '.', 'p', 't', 'h']
autoencoder-num_latents\=16-wikipedia-v2.3.4-128-finetuned-v2.5.1.5/10000.pth
['d', 'i', 'f', 'f', 'u', 's', 'i', 'o', 'n', '-', 'r', 'o', 'c', 's', 't', 'o', 'r', 'i', 'e', 's', '-', '1', '6', '-', 'd', '\\=', '5', '-', 'v', '2', '.', '3', '.', '4', '/', '1', '8', '0', '0', '0', '0', '.', 'p', 't', 'h']
diffusion-rocstories-16-d\=5-v2.3.4/180000.pth


...

/home/user/conda/lib/python3.11/site-packages/client_lib.py:412: DeprecationWarning: The `preflight_check` parameter is deprecated and has no effect. Soon the parameter support will stop completely, please do not use it.
  warnings.warn(msg, DeprecationWarning)


['a', 'u', 't', 'o', 'e', 'n', 'c', 'o', 'd', 'e', 'r', '-', 'n', 'u', 'm', '_', 'l', 'a', 't', 'e', 'n', 't', 's', '\\=', '1', '6', '-', 'w', 'i', 'k', 'i', 'p', 'e', 'd', 'i', 'a', '-', 'v', '2', '.', '3', '.', '4', '-', '1', '2', '8', '-', 'f', 'i', 'n', 'e', 't', 'u', 'n', 'e', 'd', '-', 'v', '2', '.', '5', '.', '1', '.', '9', '/', '1', '0', '0', '0', '0', '.', 'p', 't', 'h']
autoencoder-num_latents\=16-wikipedia-v2.3.4-128-finetuned-v2.5.1.9/10000.pth
['d', 'i', 'f', 'f', 'u', 's', 'i', 'o', 'n', '-', 'r', 'o', 'c', 's', 't', 'o', 'r', 'i', 'e', 's', '-', '1', '6', '-', 'd', '\\=', '5', '-', 'v', '2', '.', '3', '.', '4', '/', '1', '8', '0', '0', '0', '0', '.', 'p', 't', 'h']
diffusion-rocstories-16-d\=5-v2.3.4/180000.pth


/home/user/conda/lib/python3.11/site-packages/client_lib.py:412: DeprecationWarning: The `preflight_check` parameter is deprecated and has no effect. Soon the parameter support will stop completely, please do not use it.
  warnings.warn(msg, DeprecationWarning)


['a', 'u', 't', 'o', 'e', 'n', 'c', 'o', 'd', 'e', 'r', '-', 'n', 'u', 'm', '_', 'l', 'a', 't', 'e', 'n', 't', 's', '\\=', '1', '6', '-', 'w', 'i', 'k', 'i', 'p', 'e', 'd', 'i', 'a', '-', 'v', '2', '.', '3', '.', '4', '-', '1', '2', '8', '-', 'f', 'i', 'n', 'e', 't', 'u', 'n', 'e', 'd', '-', 'v', '2', '.', '5', '.', '1', '.', '1', '0', '/', '1', '0', '0', '0', '0', '.', 'p', 't', 'h']
autoencoder-num_latents\=16-wikipedia-v2.3.4-128-finetuned-v2.5.1.10/10000.pth
['d', 'i', 'f', 'f', 'u', 's', 'i', 'o', 'n', '-', 'r', 'o', 'c', 's', 't', 'o', 'r', 'i', 'e', 's', '-', '1', '6', '-', 'd', '\\=', '5', '-', 'v', '2', '.', '3', '.', '4', '/', '1', '8', '0', '0', '0', '0', '.', 'p', 't', 'h']
diffusion-rocstories-16-d\=5-v2.3.4/180000.pth


/home/user/conda/lib/python3.11/site-packages/client_lib.py:412: DeprecationWarning: The `preflight_check` parameter is deprecated and has no effect. Soon the parameter support will stop completely, please do not use it.
  warnings.warn(msg, DeprecationWarning)


# Old

In [20]:
script = f"{work_dir}/train_encoder.py " \
         f"dataset=rocstories " \
         f"dataset.dataset_path=/home/jovyan/vmeshchaninov/HierarchicalDiffusion/data/rocstories " \
         f"autoencoder.latent.resolutions='[1, 1, 2, 4, 8, 16]' " \
         f"encoder.model.mlm_probability=0.3 " \
         f"encoder.model.bert_masking=true " \
         f"decoder.model.mlm_probabilities='[0.7, 0.76, 0.82, 0.88, 0.94, 1.]' " \
         f"decoder.model.bert_masking=true " \
         f"autoencoder.training.batch_size=256 " \
         f"autoencoder.training.training_iters=50000 " \
         f"autoencoder.latent.is_detach_previous_latents=true " \
         f"autoencoder/optimizer=stableadam"

In [ ]:

script = f"{work_dir}/train_diffusion.py " \
         f"dataset.name='rocstories' " \
         f"dataset.dataset_path='/home/jovyan/vmeshchaninov/HierarchicalDiffusion/data/rocstories' "

In [19]:
import numpy as np

np.linspace(0.7, 1, 6)

array([0.7 , 0.76, 0.82, 0.88, 0.94, 1.  ])

In [ ]:
script = f"{work_dir}/train_encoder.py " \
         f"dataset=rocstories " \
         f"dataset.dataset_path=/home/jovyan/vmeshchaninov/HierarchicalDiffusion/data/rocstories " \
         f"autoencoder.latent.resolutions='[1, 1, 2, 4, 8]' " \
         f"encoder.model.mlm_probability=0. " \
         f"encoder.model.bert_masking=false " \
         f"decoder.model.mlm_probabilities='[1., 1., 1., 1., 1.]' " \
         f"decoder.model.bert_masking=false " \
         f"autoencoder.training.batch_size=256 " \
         f"autoencoder.training.training_iters=15000 " \
         f"autoencoder.latent.is_detach_previous_latents=true " \
         f"autoencoder/optimizer=stableadam"

In [3]:
script = f"{work_dir}/train_encoder.py " \
         f"dataset=wikipedia " \
         f"dataset.dataset_path=/home/jovyan/vmeshchaninov/HierarchicalDiffusion/data/wikipedia " \
         f"autoencoder.latent.resolutions='[16]' " \
         f"encoder.model.mlm_probability=0. " \
         f"encoder.model.bert_masking=false " \
         f"decoder.model.mlm_probabilities='[1.]' " \
         f"decoder.model.bert_masking=false " \
         f"autoencoder.training.batch_size=256 " \
         f"autoencoder.training.training_iters=50000 " \
         f"diffusion.dynamic.d=10 " \
         f"autoencoder/optimizer=stableadam"

In [13]:
def make_checkpoint(name):
    list_ = []
    for s in list(name):
        if s in ['[', ']', '(', ')', '=', ',']:
            list_.append(f"\\{s}")
            
        else:
            list_.append(s)
    t = "".join(list_)
    print(list_)
    print(t)
    return t

In [21]:
checkpoint = "autoencoder-latent_resolutions=[16]-mlm_probabilities_dec=[0.7]-wikipedia-v2.2/25000.pth"

script = f"{work_dir}/train_diffusion.py " \
         f"training=diffusion " \
         f"dataset=rocstories " \
         f"dataset.dataset_path=\"/home/jovyan/vmeshchaninov/HierarchicalDiffusion/data/rocstories\" " \
         f"autoencoder.latent.resolutions='[16]' " \
         f"autoencoder.model.load_checkpoint='{make_checkpoint(checkpoint)}' " \
         f"diffusion.dynamic.N=200 " \
         f"diffusion.training.training_iters=500000 " \
         f"diffusion.dynamic.d=7 " \
         f"diffusion/optimizer=adamw "

['a', 'u', 't', 'o', 'e', 'n', 'c', 'o', 'd', 'e', 'r', '-', 'l', 'a', 't', 'e', 'n', 't', '_', 'r', 'e', 's', 'o', 'l', 'u', 't', 'i', 'o', 'n', 's', '\\=', '\\[', '1', '6', '\\]', '-', 'm', 'l', 'm', '_', 'p', 'r', 'o', 'b', 'a', 'b', 'i', 'l', 'i', 't', 'i', 'e', 's', '_', 'd', 'e', 'c', '\\=', '\\[', '0', '.', '7', '\\]', '-', 'w', 'i', 'k', 'i', 'p', 'e', 'd', 'i', 'a', '-', 'v', '2', '.', '2', '/', '2', '5', '0', '0', '0', '.', 'p', 't', 'h']
autoencoder-latent_resolutions\=\[16\]-mlm_probabilities_dec\=\[0.7\]-wikipedia-v2.2/25000.pth


In [22]:
N_GPUS = 2
N_WORKERS = 2

base_image = "cr.ai.cloud.ru/6e6f5587-f3c6-4c5d-b535-0c4fce4c6027/jupyter-job-textdif:0.8"
EXP_NAME = 'autoencoder'


job = client_lib.Job(
    job_desc=f"#other#RnD-meshchaninov-{EXP_NAME}",
    base_image=base_image,
    script=script,
    n_workers = N_WORKERS,
    instance_type=f"a100.{N_GPUS}gpu",
    type = 'pytorch2',
    preflight_check=True,
    env_variables={
        'NCCL_IB_SL': "",
        'NCCL_P2P_LEVEL': "",
        'NCCL_NET_GDR_LEVEL': "",
        'NCCL_MAX_NRINGS': "",
        'PYTHONPATH': "",
        'PROJECT_ROOT': work_dir,
    },
    region='SR004',
    flags = {},
)

job.submit()

'Job "lm-mpi-job-3d97f4e8-3c0c-4e17-8e9a-cb25d4051772" created.'

In [11]:
job.status()

'Job status=Pending'

In [5]:
d = ""
while d != "Running": 
    d = job.status().split("=")[-1]
print(d)

Running


In [14]:
job.logs(verbose=True)

""


In [18]:
job.kill()

'Job "lm-mpi-job-5284da66-38dd-4e3f-95c3-d9a31ea456c1" deleted'

In [101]:
s = """PROJECT_ROOT=/home/jovyan/vmeshchaninov/HierarchicalDiffusion \
torchrun --nproc_per_node=2 --master_port=12345 train_diffusion.py \
training=diffusion \
dataset=rocstories \
dataset.dataset_path="/home/jovyan/vmeshchaninov/HierarchicalDiffusion/data/rocstories" \
autoencoder.model.load_checkpoint='"autoencoder-latent_dim=768-latent_resolutions=[16]-bert_model=bert-base-cased-level_weights=[1.0, 1.0, 1.0, 1.0]-bert_masking_enc=False-bert_masking_dec=False-mlm_probabilities_dec=[1.0]-optimizer=lion/20000.pth"' \
diffusion.dynamic.N=200 \
diffusion/optimizer=adamw \
autoencoder.latent.resolutions="[16]"
"""

In [102]:
print(s)

PROJECT_ROOT=/home/jovyan/vmeshchaninov/HierarchicalDiffusion torchrun --nproc_per_node=2 --master_port=12345 train_diffusion.py training=diffusion dataset=rocstories dataset.dataset_path="/home/jovyan/vmeshchaninov/HierarchicalDiffusion/data/rocstories" autoencoder.model.load_checkpoint='"autoencoder-latent_dim=768-latent_resolutions=[16]-bert_model=bert-base-cased-level_weights=[1.0, 1.0, 1.0, 1.0]-bert_masking_enc=False-bert_masking_dec=False-mlm_probabilities_dec=[1.0]-optimizer=lion/20000.pth"' diffusion.dynamic.N=200 diffusion/optimizer=adamw autoencoder.latent.resolutions="[16]"



In [103]:
str(s)

'PROJECT_ROOT=/home/jovyan/vmeshchaninov/HierarchicalDiffusion torchrun --nproc_per_node=2 --master_port=12345 train_diffusion.py training=diffusion dataset=rocstories dataset.dataset_path="/home/jovyan/vmeshchaninov/HierarchicalDiffusion/data/rocstories" autoencoder.model.load_checkpoint=\'"autoencoder-latent_dim=768-latent_resolutions=[16]-bert_model=bert-base-cased-level_weights=[1.0, 1.0, 1.0, 1.0]-bert_masking_enc=False-bert_masking_dec=False-mlm_probabilities_dec=[1.0]-optimizer=lion/20000.pth"\' diffusion.dynamic.N=200 diffusion/optimizer=adamw autoencoder.latent.resolutions="[16]"\n'